In [ ]:
# @title 1. Initialize & Connect to Drive (Step 1)🚀
import os
import sys
import logging

# --- 🎨 Color & Emoji Class ---
class Style:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    RESET = '\033[0m'
    BOLD = '\033[1m'

print(f"{Style.HEADER}{Style.BOLD}🔄 Initializing Environment...{Style.RESET}")

# --- Install Dependencies ---
try:
    import yt_dlp
    print(f"{Style.GREEN}✅ yt-dlp is already installed.{Style.RESET}")
except ImportError:
    print(f"{Style.CYAN}⬇️  Installing yt-dlp...{Style.RESET}")
    !pip install yt_dlp -q
    import yt_dlp
    print(f"{Style.GREEN}✅ Installation complete.{Style.RESET}")

# --- Mount Google Drive ---
from google.colab import drive

def mount_drive():
    drive_path = '/content/drive'
    if os.path.exists(drive_path):
        print(f"{Style.GREEN}✅ Google Drive is already mounted.{Style.RESET}")
        return drive_path

    try:
        print(f"{Style.YELLOW}📂 Please authorize Google Drive access...{Style.RESET}")
        drive.mount(drive_path)
        print(f"{Style.GREEN}✅ Drive mounted successfully!{Style.RESET}")
        return drive_path
    except Exception as e:
        print(f"{Style.RED}❌ Failed to mount Drive.{Style.RESET}")
        sys.exit("Critical Error: Cannot proceed without Drive.")

# Execute Mount
DRIVE_ROOT = mount_drive()
DOWNLOAD_FOLDER = os.path.join(DRIVE_ROOT, "My Drive", "YouTube_Downloads")

# Create folder if missing
if not os.path.exists(DOWNLOAD_FOLDER):
    try:
        os.makedirs(DOWNLOAD_FOLDER)
        print(f"{Style.CYAN}📁 Created folder: {DOWNLOAD_FOLDER}{Style.RESET}")
    except OSError as e:
        print(f"{Style.RED}❌ Error creating folder.{Style.RESET}")

In [ ]:
# @title 2. Download Video (Step 2 Select Quality) 🎬
# @markdown Select your desired quality, enter the URL, and press Play.

target_url = "https://youtube.com/video/ . . ." # @param {type:"string"}
video_quality = "720p" # @param ["720p", "1080p" ,"1440p"]

import yt_dlp
import os
import sys

# --- 🎨 Style Definitions ---
class Style:
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BLUE = '\033[94m'
    RESET = '\033[0m'
    BOLD = '\033[1m'

# --- 📊 Custom Progress Hook ---
def progress_hook(d):
    if d['status'] == 'downloading':
        total = d.get('total_bytes') or d.get('total_bytes_estimate', 0)
        downloaded = d.get('downloaded_bytes', 0)

        if total > 0:
            percent = downloaded / total * 100
        else:
            percent = 0

        bar_length = 25
        filled_length = int(bar_length * percent // 100)
        bar = '▓' * filled_length + '░' * (bar_length - filled_length)

        speed = d.get('_speed_str', 'N/A')
        eta = d.get('_eta_str', 'N/A')

        sys.stdout.write(
            f"\r{Style.CYAN}🚀 {Style.BOLD}{percent:.1f}%{Style.RESET} "
            f"|{Style.GREEN}{bar}{Style.RESET}| "
            f"[{Style.YELLOW}{speed}{Style.RESET}] "
            f"[{Style.BLUE}ETA: {eta}{Style.RESET}]"
        )
        sys.stdout.flush()

    elif d['status'] == 'finished':
        sys.stdout.write(f"\n{Style.GREEN}✨ Download Complete! Merging Audio & Video...{Style.RESET}\n")

# --- ⚙️ Main Logic ---
def get_format_string(quality):
    """Returns the yt-dlp format string based on user selection."""
    # ALWAYS adds +bestaudio to ensure top tier sound
    if quality == "1080p":
        # Try 1080p first, fallback to best available below 1080p
        return 'bestvideo[height=1080]+bestaudio/bestvideo[height<=1080]+bestaudio/best[height<=1080]/best'
    else:
        # Default to 720p
        return 'bestvideo[height=720]+bestaudio/bestvideo[height<=720]+bestaudio/best[height<=720]/best'

def download_video_secure(url, output_path, quality):
    if not url or "youtube.com" not in url and "youtu.be" not in url:
        print(f"{Style.RED}❌ Invalid URL provided.{Style.RESET}")
        return

    print(f"\n{Style.CYAN}🔍 Processing: {Style.BOLD}{url}{Style.RESET}")
    print(f"{Style.BLUE}🎚️  Selected Quality: {Style.BOLD}{quality} + Best Audio{Style.RESET}")

    ydl_opts = {
        'format': get_format_string(quality),
        'merge_output_format': 'mp4',
        'outtmpl': f'{output_path}/%(title)s [%(id)s].%(ext)s',
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
        'progress_hooks': [progress_hook]
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print(f"{Style.YELLOW}📡 Fetching metadata...{Style.RESET}")
            info = ydl.extract_info(url, download=False)
            title = info.get('title', 'Unknown')
            print(f"{Style.CYAN}🎞️  Title: {Style.BOLD}{title}{Style.RESET}")
            print(f"{Style.YELLOW}⬇️  Starting download...{Style.RESET}")

            ydl.download([url])

            print(f"\n{Style.GREEN}{Style.BOLD}✅ ALL DONE!{Style.RESET}")
            print(f"{Style.GREEN}💾 Saved to: {Style.RESET}{output_path}")

    except yt_dlp.utils.DownloadError as e:
        print(f"\n{Style.RED}❌ Download Failed.{Style.RESET}")
        print(f"{Style.RED}Error details: {e}{Style.RESET}")
    except Exception as e:
        print(f"\n{Style.RED}❌ An unexpected error occurred.{Style.RESET}")
        print(f"{Style.RED}{str(e)}{Style.RESET}")

# --- 🚀 Execution ---
if 'DOWNLOAD_FOLDER' not in globals():
    # Emergency fallback if Cell 1 wasn't run
    DOWNLOAD_FOLDER = "/content/drive/My Drive/YouTube_Downloads"
    if not os.path.exists("/content/drive"):
         print(f"{Style.RED}⚠️ Drive not connected! Please run Cell 1 first.{Style.RESET}")
    else:
         download_video_secure(target_url, DOWNLOAD_FOLDER, video_quality)
else:
    download_video_secure(target_url, DOWNLOAD_FOLDER, video_quality)